<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-데이터-전처리" data-toc-modified-id="1.-데이터-전처리-1">1. 데이터 전처리</a></span></li><li><span><a href="#2.-카카오-API-활용---위도,-경도-변환" data-toc-modified-id="2.-카카오-API-활용---위도,-경도-변환-2">2. 카카오 API 활용 - 위도, 경도 변환</a></span></li></ul></div>

서울시 착한가격업소 현황 : https://data.seoul.go.kr/dataList/OA-1173/S/1/datasetView.do

서울시 가격안정 모범업소 상품목록 현황 : https://data.seoul.go.kr/dataList/OA-1174/S/1/datasetView.do

In [1]:
import requests
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas

import warnings
warnings.filterwarnings(action='ignore')

# 1. 데이터 전처리

In [2]:
data = pd.read_csv('서울시 착한가격업소 현황.csv', encoding='cp949')
print(data.shape)
data.head()

(1031, 12)


,업소아이디,업소명,분류코드,분류코드명,업소 주소,업소 전화번호,찾아오시는 길,업소정보,자랑거리,추천수,기준년월,업소 사진
0,6017,아폴로헤어크리닉,5,이 미용업,서울특별시 중랑구 사가정로50길 67 (면목동),495-7337,면목골목시장 65호,영업시간 :09:00~21:00 좌석수 :5 휴무일 :매주 수요일 예약 :불가능 카...,면목시장내 위치 형편이 어려우신 분은 요금을 할인해주며 20년간 봉사활동을 하시는 ...,450,21-2,http://tearstop.seoul.go.kr/mulga/photo/201511...
1,6015,상록수미용실,5,이 미용업,서울특별시 중랑구 동일로99길 22 (면목동),NaN,면목2동 다울아파트에서 동일로로 가는 골목에 위치,영업시간 : 09:30~21:00 좌석수 : 3 휴무일 : 매주 일요일 예약 :불가...,주 고객이 어르신들이라 가격을 저렴하게 책정하여 파마 14000원 등 파격적인 요금...,432,21-2,http://tearstop.seoul.go.kr/mulga/photo/201511...
2,4506,열린미용실,5,이 미용업,서울특별시 관악구 난곡로40길 16 (신림동),-,미성동 노선버스 신일교회앞 하차,영업시간 : 오전 9시 ~ 오후 8시 휴무일 : 매주 일요일 예약 : 가능 카드 :...,▷관악구 평균가격보다 20%이상 저렴. ▷경로자 장애인 할인해 드림 ▷한달에 한번씩...,336,21-2,http://tearstop.seoul.go.kr/mulga/photo/b57.jpg
3,3002,미림17분칼라,13,기타서비스업종,서울특별시 관악구 호암로26길 1 (신림동),871-7070,미림여고 입구 국민은행 옆 위치,영업시간 :오전 9시 30분 ~ 오후 9시 30분 휴무일 : 매월 둘째 넷째주 일...,▷관악구 평균가격(인화료 -3*5cm) 보다 20% 저렴 ▷ 경로자 20%할인,318,21-2,http://tearstop.seoul.go.kr/mulga/photo/b46.jpg
4,4242,덕성이발관,5,이 미용업,서울특별시 관악구 장군봉2길 11 (봉천동),-,봉천역 3번 출구에서 시장골목으로 100미터,영업시간 : 오전 7시 ~ 오후 9시 좌석수 : 4 휴무일 : 매주 화요일 주차 :...,▷관악구 평균가격보다 10%이상 저렴 ▷시설이 세련되지는 않았지만 동네 사랑방 같은...,264,21-2,http://tearstop.seoul.go.kr/mulga/photo/201204...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031 entries, 0 to 1030
Data columns (total 12 columns):
업소아이디      1031 non-null int64
업소명        1031 non-null object
분류코드       1031 non-null int64
분류코드명      1031 non-null object
업소 주소      1031 non-null object
업소 전화번호    992 non-null object
찾아오시는 길    484 non-null object
업소정보       1000 non-null object
자랑거리       777 non-null object
추천수        1031 non-null int64
기준년월       1031 non-null object
업소 사진      1031 non-null object
dtypes: int64(3), object(9)
memory usage: 96.8+ KB


In [4]:
# 시군구 컬럼 추가
data['시군구'] = data['업소 주소']
data['시군구'] = data['시군구'].apply(lambda x: [i for i in x.split(' ') if '구' in i])

In [5]:
import numpy as np

for i in range(len(data)):
    try:
        data['시군구'][i] = data['시군구'][i][0]
    except:
        data['시군구'][i] = np.nan

In [6]:
data['시군구'].unique()

array(['중랑구', '관악구', '송파구', '강서구', '양천구', '구로구', '광진구', '중구', '동대문구',
       '노원구', '종로구', '용산구', '강동구', '영등포구', '성북구', '마포구', '강북구', '서대문구',
       '서초구', '도봉구', '성동구', '금천구', '강남구', nan, '은평구',
       '노원구\t한글비석로444(교립프라자', '구로동로', '(구로동)', '동작구'], dtype=object)

In [7]:
data[(data.시군구=='노원구\t한글비석로444(교립프라자')|(data.시군구=='구로동로')|(data.시군구=='(구로동)')]

,업소아이디,업소명,분류코드,분류코드명,업소 주소,업소 전화번호,찾아오시는 길,업소정보,자랑거리,추천수,기준년월,업소 사진,시군구
260,8228,세탁편의점,7,세탁업,서울특별시 노원구\t한글비석로444(교립프라자 1층) (상계동),02-3391-4459,NaN,이용시간: 08:30 ~ 20:00 주차 : 가능 배달 : 가능 휴무일 : 매주일요...,세탁편의점이지만 아파트밀집지역에 위치해 있으므로 세탁물 수거와 배달서비스를 하고 있...,3,21-2,http://tearstop.seoul.go.kr/mulga/photo/201401...,노원구\t한글비석로444(교립프라자
436,8950,용갈비,1,한식,서울특별시 구로동로 96-4 (구로동),02-869-2442,NaN,NaN,NaN,2,21-2,http://tearstop.seoul.go.kr/mulga/photo/201508...,구로동로
532,8953,포토베베스튜디오,13,기타서비스업종,서울특별시 도림로12길 13 (구로동),02-862-1050,NaN,null,NaN,2,21-2,http://tearstop.seoul.go.kr/mulga/photo/201508...,(구로동)


In [8]:
data.시군구[260] = '노원구'
data.시군구[436] = '구로구'
data.시군구[532] = '구로구'

In [9]:
data[data.시군구.isnull()]

,업소아이디,업소명,분류코드,분류코드명,업소 주소,업소 전화번호,찾아오시는 길,업소정보,자랑거리,추천수,기준년월,업소 사진,시군구
151,6052,소문난맛함흥냉면,1,한식,서울특별시 무학봉16길 1 (하왕십리동),2299-5537,전풍호텔 건너편에서 홈플러스쪽으로 200m 정도 앞으로 오시면 됩니다.(왕금당 맞은편),영업시간 : 11:00~24:00 좌석수 : 16석 휴무일 : 없음 배달 : x ...,주인이 거래처를 통해 식자재 직접 조달 질좋은 고기사용,5,21-2,http://tearstop.seoul.go.kr/mulga/photo/201308...,NaN
441,8807,신 쿡(cook),1,한식,서울특별시 중랑천로10길 39,02-439-2012,면목2동 서울우유 본사 맞은편,영업시간 : 11:00 ~ 21:30 좌석수 : 18 휴무일 : 매주 화요일 배달 ...,작지만 카페같은 실내분위기에 샐러드 피자 스파게티와 세트메뉴까지 맛있는 요리를 비교...,2,21-2,http://tearstop.seoul.go.kr/mulga/photo/201511...,NaN
571,9508,금호동바다장어초이408,1,한식,서울특별시 독서당로 295-5(금호동3가),02-2231-1733,NaN,영업시간 : 좌석수 : 휴무일 : 배달 : 예약 : 카드 : 주차 : 홈페이지 주소...,개업(2017.12월)이후 현재까지 가격동결,0,21-2,http://tearstop.seoul.go.kr/mulga/photo/202108...,NaN
580,9423,다담2900,1,한식,서울특별시 망우로66길 75 1층,02-493-3339,NaN,영업시간 : 09 ~ 21 좌석수 : 휴무일 : 배달 : 예약 : 카드 : 주차 :...,NaN,0,21-2,http://tearstop.seoul.go.kr/mulga/photo/,NaN
636,9347,밥리버거 앤 치킨호프,1,한식,서울특별시 언주로 405,02-552-0388,NaN,영업시간 :06:30~20:00 좌석수 : 휴무일 : 배달 : 예약 : 카드 : 주...,아침 식사 못하고 출근하는 직장인들 위해 저렴한 음식제공,0,21-2,http://tearstop.seoul.go.kr/mulga/photo/201909...,NaN
642,9377,선우동가게,1,한식,서울특별시 남부순환로359길 10 지하1층,02-572-8887,NaN,영업시간 :11:00~21:30 좌석수 : 휴무일 : 배달 :불가능 예약 : 카드 ...,NaN,0,21-2,http://tearstop.seoul.go.kr/mulga/photo/202005...,NaN
657,9346,대풍참숯갈비,1,한식,서울특별시 논현로76길 8,02-564-6563,NaN,영업시간 : 07:00~23:00 좌석수 : 휴무일 : 배달 : 예약 : 카드 : ...,손님들이 저렴한 가격에 맛있게 식사하고 즐거워할 때 보람을 느낌,0,21-2,http://tearstop.seoul.go.kr/mulga/photo/201909...,NaN
670,9442,테스트,1,한식,서울특별시 테스트,123-4567,NaN,영업시간 : 좌석수 : 휴무일 : 배달 : 예약 : 카드 : 주차 : 홈페이지 주소...,NaN,0,21-2,http://tearstop.seoul.go.kr/mulga/photo/,NaN
751,9353,착한미용실,5,이 미용업,서울특별시 언주로67길 19 1층 103호,02-558-1706,NaN,영업시간 :10:30~18:00 좌석수 : 휴무일 : 배달 : 예약 : 카드 : 주...,약이 좋으면서도 가격이 저렴하여 머릿결이 상하지 않도록 최선을 다함. 영양공급도 추...,0,21-2,http://tearstop.seoul.go.kr/mulga/photo/,NaN
755,9153,커피디인,4,기타외식업(다방패스트푸드등),서울특별시 거북골로20길3,309-1688,NaN,영업시간 : 좌석수 : 휴무일 : 배달 : 예약 : 카드 : 주차 : 홈페이지 주소...,NaN,0,21-2,http://tearstop.seoul.go.kr/mulga/photo/201809...,NaN


In [10]:
data.시군구[151] = '성동구'
data.시군구[441] = '중랑구'
data.시군구[571] = '성동구'
data.시군구[580] = '중랑구'
data.시군구[636] = '강남구'
data.시군구[642] = '강남구'
data.시군구[657] = '강남구'

data.drop([670], axis=0, inplace=True)

data.시군구[751] = '강남구'
data.시군구[755] = '서대문구'
data.시군구[777] = '서대문구'
data.시군구[823] = '강남구'
data.시군구[832] = '성북구'
data.시군구[876] = '강남구'
data.시군구[879] = '강남구'
data.시군구[884] = '강남구'
data.시군구[887] = '성동구'
data.시군구[898] = '서대문구'
data.시군구[956] = '강남구'
data.시군구[989] = '강남구'
data.시군구[1024] = '송파구'
data.시군구[1025] = '송파구'
data.시군구[1026] = '송파구'

In [12]:
data.reset_index(drop=True, inplace=True)

In [15]:
data['업소 주소'] = data['업소 주소'].apply(lambda x: x.split('(')[0].strip())
add_list = data['업소 주소'].tolist()
add_list[:5]

['서울특별시 중랑구 사가정로50길 67',
 '서울특별시 중랑구 동일로99길 22',
 '서울특별시 관악구 난곡로40길 16',
 '서울특별시 관악구 호암로26길 1',
 '서울특별시 관악구 장군봉2길 11']

# 2. 카카오 API 활용 - 위도, 경도 변환

In [16]:
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?&query=" + address
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK *****"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['address_name'], document['y'], document['x']]
        except:
            val = [0, 0, 0]
    return val

In [17]:
# 주소 검색 함수 적용
address_to_latlon('서울특별시 중랑구 사가정로50길 67')

['서울 중랑구 사가정로50길 67', '37.5778510022974', '127.085011397946']

In [18]:
# 주소 검색 함수 반복 적용
list1 = []
for i in add_list:
    list1.append(address_to_latlon(i))

In [19]:
data2 = pd.DataFrame(list1, columns=['주소','위도','경도']).drop('주소', axis=1)
data2.head()

,위도,경도
0,37.5778510022974,127.085011397946
1,37.587922318728,127.078621025398
2,37.4752814192096,126.917677423159
3,37.4709918069211,126.933898672188
4,37.4824466875174,126.939594879058


In [20]:
data3 = pd.concat([data, data2], axis=1)
print(data3.shape)
data3.head()

(1030, 15)


,업소아이디,업소명,분류코드,분류코드명,업소 주소,업소 전화번호,찾아오시는 길,업소정보,자랑거리,추천수,기준년월,업소 사진,시군구,위도,경도
0,6017,아폴로헤어크리닉,5,이 미용업,서울특별시 중랑구 사가정로50길 67,495-7337,면목골목시장 65호,영업시간 :09:00~21:00 좌석수 :5 휴무일 :매주 수요일 예약 :불가능 카...,면목시장내 위치 형편이 어려우신 분은 요금을 할인해주며 20년간 봉사활동을 하시는 ...,450,21-2,http://tearstop.seoul.go.kr/mulga/photo/201511...,중랑구,37.5778510022974,127.085011397946
1,6015,상록수미용실,5,이 미용업,서울특별시 중랑구 동일로99길 22,NaN,면목2동 다울아파트에서 동일로로 가는 골목에 위치,영업시간 : 09:30~21:00 좌석수 : 3 휴무일 : 매주 일요일 예약 :불가...,주 고객이 어르신들이라 가격을 저렴하게 책정하여 파마 14000원 등 파격적인 요금...,432,21-2,http://tearstop.seoul.go.kr/mulga/photo/201511...,중랑구,37.587922318728,127.078621025398
2,4506,열린미용실,5,이 미용업,서울특별시 관악구 난곡로40길 16,-,미성동 노선버스 신일교회앞 하차,영업시간 : 오전 9시 ~ 오후 8시 휴무일 : 매주 일요일 예약 : 가능 카드 :...,▷관악구 평균가격보다 20%이상 저렴. ▷경로자 장애인 할인해 드림 ▷한달에 한번씩...,336,21-2,http://tearstop.seoul.go.kr/mulga/photo/b57.jpg,관악구,37.4752814192096,126.917677423159
3,3002,미림17분칼라,13,기타서비스업종,서울특별시 관악구 호암로26길 1,871-7070,미림여고 입구 국민은행 옆 위치,영업시간 :오전 9시 30분 ~ 오후 9시 30분 휴무일 : 매월 둘째 넷째주 일...,▷관악구 평균가격(인화료 -3*5cm) 보다 20% 저렴 ▷ 경로자 20%할인,318,21-2,http://tearstop.seoul.go.kr/mulga/photo/b46.jpg,관악구,37.4709918069211,126.933898672188
4,4242,덕성이발관,5,이 미용업,서울특별시 관악구 장군봉2길 11,-,봉천역 3번 출구에서 시장골목으로 100미터,영업시간 : 오전 7시 ~ 오후 9시 좌석수 : 4 휴무일 : 매주 화요일 주차 :...,▷관악구 평균가격보다 10%이상 저렴 ▷시설이 세련되지는 않았지만 동네 사랑방 같은...,264,21-2,http://tearstop.seoul.go.kr/mulga/photo/201204...,관악구,37.4824466875174,126.939594879058


In [21]:
data3.to_excel('서울시 착한가격업소 현황 (좌표추가).xlsx', index=False, encoding='cp949')